In [33]:
import numpy as np
import cv2
import os

def add_gaussian_noise(image):
    row, col, ch = image.shape
    mean = 0
    var = 0.01
    sigma = var ** 0.5
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    noisy = image + gauss * 255 * 0.1
    return np.clip(noisy, 0, 255).astype('uint8')

def add_salt_and_pepper_noise(image):
    row, col, ch = image.shape
    s_vs_p = 0.5
    amount = 0.1
    out = np.copy(image)
    num_salt = np.ceil(amount * image.size * s_vs_p)
    coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape]
    out[coords[0], coords[1], :] = 1
    num_pepper = np.ceil(amount * image.size * (1. - s_vs_p))
    coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape]
    out[coords[0], coords[1], :] = 0
    return out

def add_speckle_noise(image):
    row, col, ch = image.shape
    gauss = np.random.randn(row, col, ch)
    gauss = gauss.reshape(row, col, ch)        
    noisy = image + image * gauss * 0.1
    return np.clip(noisy, 0, 255).astype('uint8')

def add_poisson_noise(image):
    scale = 255.0 / image.max()
    data = image * scale
    noisy_data = np.random.poisson(data) / scale
    noisy_image = np.clip(noisy_data, 0, 255).astype('uint8')
    return noisy_image

def process_images(root_directory, output_directory):
    # Create output directories for each type of noise
    for noise_type in ['gaussian', 'salt_pepper', 'speckle', 'poisson']:
        os.makedirs(os.path.join(output_directory, noise_type), exist_ok=True)

    # Process each file in the directory recursively
    for subdir, dirs, files in os.walk(root_directory):
        for filename in files:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                filepath = os.path.join(subdir, filename)
                image = cv2.imread(filepath)
                if image is not None:
                    # Apply each type of noise and save
                    noisy_images = {
                        'gaussian': add_gaussian_noise(image),
                        'salt_pepper': add_salt_and_pepper_noise(image),
                        'speckle': add_speckle_noise(image),
                        'poisson': add_poisson_noise(image)
                    }
                    for noise_type, noisy_image in noisy_images.items():
                        output_path = os.path.join(output_directory, noise_type, filename)
                        cv2.imwrite(output_path, noisy_image)
                    print(f"Processed and saved all noise types for {filename}")



In [41]:
root_directory = r'C:\Users\Qwerty\Downloads\Benchmarks Dataset\DIV2K\data'
output_directory = r'C:\Users\Qwerty\Downloads\Benchmarks Dataset\DIV2K\noisy'
process_images(root_directory, output_directory)

Processed and saved all noise types for 0001.png
Processed and saved all noise types for 0002.png
Processed and saved all noise types for 0003.png
Processed and saved all noise types for 0004.png
Processed and saved all noise types for 0005.png
Processed and saved all noise types for 0006.png
Processed and saved all noise types for 0007.png
Processed and saved all noise types for 0008.png
Processed and saved all noise types for 0009.png
Processed and saved all noise types for 0010.png
Processed and saved all noise types for 0011.png
Processed and saved all noise types for 0012.png
Processed and saved all noise types for 0013.png
Processed and saved all noise types for 0014.png
Processed and saved all noise types for 0015.png
Processed and saved all noise types for 0016.png
Processed and saved all noise types for 0017.png
Processed and saved all noise types for 0018.png
Processed and saved all noise types for 0019.png
Processed and saved all noise types for 0020.png
Processed and saved 

In [22]:
import numpy as np
import cv2

def calculate_snr(image):
    """Calculate Signal to Noise Ratio (SNR) for a given image."""
    mean_signal = np.mean(image)
    std_deviation = np.std(image)
    snr = 20 * np.log10(mean_signal / std_deviation)
    return snr

def main(original_image_path, noisy_image_path):
    # Load the original and noisy images in grayscale
    original_image = cv2.imread(original_image_path, cv2.IMREAD_GRAYSCALE)
    noisy_image = cv2.imread(noisy_image_path, cv2.IMREAD_GRAYSCALE)

    if original_image is None or noisy_image is None:
        print("Failed to load one or both images.")
        return

    # Calculate SNR for both images
    snr_original = calculate_snr(original_image)
    snr_noisy = calculate_snr(noisy_image)

    print(f"Original Image SNR: {snr_original:.2f} dB")
    print(f"Noisy Image SNR: {snr_noisy:.2f} dB")

    # Compare SNR values
    if snr_original > snr_noisy:
        print("The original image has a higher SNR.")
    elif snr_original < snr_noisy:
        print("The noisy image has a higher SNR.")
    else:
        print("Both images have the same SNR.")

# Provide the path to your images
original_image_path = r'C:\Users\Qwerty\Downloads\Benchmarks Dataset\CIFAR-10\extracted\data\frog/image_2409.png'
noisy_image_path = r'C:\Users\Qwerty\Downloads\Benchmarks Dataset\CIFAR-10\noisy\poisson/image_2409.png'
main(original_image_path, noisy_image_path)


Original Image SNR: 12.94 dB
Noisy Image SNR: 12.93 dB
The original image has a higher SNR.
